In [ ]:
%load_ext autoreload
%autoreload 2

import bnl
import frameless_eval as fle
import numpy as np

In [ ]:
# create two hierarchies:
seg1 = bnl.S.from_bs([0, 2.5, 6], ["A", "B"])
seg2 = bnl.S.from_bs([0, 1.2, 2.5, 3.5, 6], ["a", "b", "c", "b"])
seg3 = bnl.S.from_bs([0, 1.2, 4, 6], ["Mi", "Re", "Do"])
seg4 = bnl.S.from_bs([0, 1.2, 3, 4, 6], ["T", "PD", "D", "T"])
seg5 = bnl.S.from_bs([0, 1.2, 2, 3, 4, 4.7, 4.9, 6], ["I", "IV", "ii", "V", "I", "IV", "I"])


est = bnl.MS([seg1, seg2], name="estimate").prune_layers()
ref = bnl.MS([seg3, seg4, seg5], name='reference').prune_layers()

In [ ]:
# Show two Hierarchies
from plotly.subplots import make_subplots

def plot_two_hierarchies(ref, est, subplot_titles=None):
    fig_est = est.plot()
    fig_ref = ref.plot()

    fig = make_subplots(
        rows=2, cols=1, shared_xaxes=True, row_heights=[3, 2],
        vertical_spacing=0.2, subplot_titles=subplot_titles
    )

    for tr in fig_ref.data:
        fig.add_trace(tr, row=1, col=1)
    for tr in fig_est.data:
        fig.add_trace(tr, row=2, col=1)


    fig.update_layout(
        height=250, width=600,showlegend=False, barmode="overlay", 
        xaxis2_range=[0, 6], xaxis2_title="Time (sec)", margin=dict(t=30, l=10, r=10, b=60),
        yaxis1=dict(
            categoryorder="array",
            categoryarray=[layer.name for layer in reversed(ref)],
        ),
        yaxis2=dict(
            categoryorder="array",
            categoryarray=[layer.name for layer in reversed(est)],
        ),
    )
    return fig



In [ ]:
def plot_two_segs(ref, est, subplot_titles=None):
    fig_est = est.plot()
    fig_ref = ref.plot()

    fig = make_subplots(
        rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1, subplot_titles=subplot_titles
    )
    
    for tr in fig_ref.data:
        fig.add_trace(tr, row=1, col=1)
    for tr in fig_est.data:
        fig.add_trace(tr, row=2, col=1)

    fig.update_layout(
        height=150, width=400, showlegend=False, barmode="overlay",
        xaxis2_range=[0, 6], margin=dict(t=30, l=10, r=10, b=10),
    )
    fig.update_yaxes(visible=False)

    return fig


In [ ]:
ref = bnl.S.from_bs([0, 1.25, 3.05, 4.02, 6], ["T", "PD", "D", "T"], name="ref")
est = bnl.S.from_bs([0, 1.25, 2.55, 3.55, 6], ["a", "b", "c", "b"], name ='est')

ms = bnl.MS([ref, est], name="Labeled Segmentations to Compare")
fig = ms.plot()
fig.write_image("tn1.pdf", width=450, height=180)

In [ ]:
#frame based
frame_center = np.arange(0.1, 6, 0.2)
frame_itvls = np.array(list(zip(frame_center - 0.1, frame_center + 0.1)))

frame_labels = fle.utils.labels_at_ts(ms.itvls, ms.labels, frame_center)
ref_frame = bnl.S.from_itvls(frame_itvls, frame_labels[0], name="ref")
est_frame = bnl.S.from_itvls(frame_itvls, frame_labels[1], name="est")
frame_ms = bnl.MS([ref_frame, est_frame], name="Frames-based (0.2 sec) alignment")
fig = frame_ms.plot()
fig.write_image("tn2.pdf", width=450, height=180)


In [ ]:
#event based
event_itvls, event_labels_est, event_labels_ref = fle.utils.make_common_itvls(
    [est.itvls], [est.labels], [ref.itvls], [ref.labels]
)

ref_event = bnl.S.from_itvls(event_itvls, event_labels_ref[0], name="ref")
est_event = bnl.S.from_itvls(event_itvls, event_labels_est[0], name="est")
event_ms = bnl.MS([ref_event, est_event], name="Event-based (exact) alignment")
fig = event_ms.plot()
fig.write_image("tn3.pdf", width=450, height=180)

In [ ]:
fig = plot_two_hierarchies(ref, est, ["Reference (original, boundaries unaligned)", "Estimate (original, boundaries unaligned)"])
fig.write_image("thumbnail_setup.pdf", width=600, height=225)


In [ ]:
# Show framebased Chopping

frame_center = np.arange(0.1, 6, 0.2)
frame_itvls = np.array(list(zip(frame_center - 0.1, frame_center + 0.1)))

frame_labels_est = fle.utils.labels_at_ts(est.itvls, est.labels, frame_center)
frame_labels_ref = fle.utils.labels_at_ts(ref.itvls, ref.labels, frame_center)
frame_est = bnl.MS.from_itvls([frame_itvls]*2, frame_labels_est)
frame_ref = bnl.MS.from_itvls([frame_itvls]*3, frame_labels_ref)

fig = plot_two_hierarchies(frame_ref, frame_est, ["Reference (frame-based alignment)", "Estimate (frame-based alignment)"])
fig.write_image("thumbnail_frame.pdf", width=600, height=250)


In [ ]:
event_itvls, event_labels_est, event_labels_ref = fle.utils.make_common_itvls(
    est.itvls, est.labels, ref.itvls, ref.labels
)
event_est = bnl.MS.from_itvls([event_itvls] * 2, event_labels_est)
event_ref = bnl.MS.from_itvls([event_itvls] * 3, event_labels_ref)

fig = plot_two_hierarchies(event_ref, event_est, ["Reference (event-based alignment)", "Estimate (event-based alignment)"])
fig.write_image("thumbnail_event.pdf", width=600, height=250)

